In [1]:
from ftplib import FTP
import copy

import numpy as np

from astropy.coordinates import Angle
from astropy.time import Time, TimeDelta
from astropy import units as u

from secret_key import login, password
from BSA3_pack import *

In [2]:
"""
N1 – 194.149.67.97 (от 20° до 42°)
N2 – 194.149.67.38 (от -9° до 20°)
"""
adress = '194.149.67.97'
con = FTP(adress, login, password)

In [3]:
con.cwd('2018')
con.cwd('may2018')

'250 CWD command successful.'

In [4]:
file_list = con.nlst('*.pnt')

In [5]:
def read_pnt_header(line):
    header_pnt.append(line.decode("utf-8"))

In [7]:
pulsar_ra = Angle("12:39:40.4614", unit='hourangle')
file_with_pulsar = []
for file in file_list:
    con = FTP(adress, login, password)
    con.cwd('2018')
    con.cwd('may2018')
    header_pnt = []
    try:
        con.retrbinary('RETR ' + file, callback=read_pnt_header, blocksize=493)
    except UnicodeDecodeError:
        print('Read was done ' + file + '!')
    except ConnectionResetError:
        print('!!! Connection was aborted !!!')
        
    header = {}
    for line in header_pnt[0].split('\n'):
        try:
            a, *b = line.split()
            header[a] = b
        except ValueError:
            header['Finish'] = ['Finish']
        
    t_b, t_e = get_time_begin_and_end(header)
    sid_t_b = my_sidereal_time(t_b)
    sid_t_e = my_sidereal_time(t_e - 4*u.hour)
    
    if sid_t_b < pulsar_ra < sid_t_e:
        file_with_pulsar.append(file)

Read was done 010518_00_N1_00.pnt!
Read was done 010518_01_N1_00.pnt!
Read was done 010518_02_N1_00.pnt!
Read was done 010518_03_N1_00.pnt!
Read was done 010518_04_N1_00.pnt!
Read was done 010518_05_N1_00.pnt!
Read was done 010518_06_N1_00.pnt!
Read was done 010518_07_N1_00.pnt!
Read was done 010518_08_N1_00.pnt!
Read was done 010518_09_N1_00.pnt!
Read was done 010518_10_N1_00.pnt!
Read was done 010518_11_N1_00.pnt!
Read was done 010518_12_N1_00.pnt!
Read was done 010518_13_N1_00.pnt!
Read was done 010518_14_N1_00.pnt!
Read was done 010518_15_N1_00.pnt!
Read was done 010518_16_N1_00.pnt!
Read was done 010518_17_N1_00.pnt!
Read was done 010518_18_N1_00.pnt!
Read was done 010518_19_N1_00.pnt!
Read was done 010518_20_N1_00.pnt!
Read was done 010518_21_N1_00.pnt!
Read was done 010518_22_N1_00.pnt!
Read was done 010518_23_N1_00.pnt!
Read was done 020518_00_N1_00.pnt!
Read was done 020518_01_N1_00.pnt!
Read was done 020518_02_N1_00.pnt!
Read was done 020518_03_N1_00.pnt!
Read was done 020518

Read was done 100518_18_N1_00.pnt!
Read was done 100518_19_N1_00.pnt!
Read was done 100518_20_N1_00.pnt!
Read was done 100518_21_N1_00.pnt!
Read was done 100518_22_N1_00.pnt!
Read was done 100518_23_N1_00.pnt!
Read was done 110518_00_N1_00.pnt!
Read was done 110518_01_N1_00.pnt!
Read was done 110518_02_N1_00.pnt!
Read was done 110518_03_N1_00.pnt!
Read was done 110518_04_N1_00.pnt!
Read was done 110518_05_N1_00.pnt!
Read was done 110518_06_N1_00.pnt!
Read was done 110518_07_N1_00.pnt!
Read was done 110518_08_N1_00.pnt!
Read was done 110518_09_N1_00.pnt!
Read was done 110518_10_N1_00.pnt!
Read was done 110518_11_N1_00.pnt!
Read was done 110518_12_N1_00.pnt!
Read was done 110518_13_N1_00.pnt!
Read was done 110518_14_N1_00.pnt!
Read was done 110518_15_N1_00.pnt!
Read was done 110518_16_N1_00.pnt!
Read was done 110518_17_N1_00.pnt!
Read was done 110518_18_N1_00.pnt!
Read was done 110518_19_N1_00.pnt!
Read was done 110518_20_N1_00.pnt!
Read was done 110518_21_N1_00.pnt!
Read was done 110518

Read was done 200518_13_N1_00.pnt!
Read was done 200518_14_N1_00.pnt!
Read was done 200518_15_N1_00.pnt!
Read was done 200518_16_N1_00.pnt!
Read was done 200518_17_N1_00.pnt!
Read was done 200518_18_N1_00.pnt!
Read was done 200518_19_N1_00.pnt!
Read was done 200518_20_N1_00.pnt!
Read was done 200518_21_N1_00.pnt!
Read was done 200518_22_N1_00.pnt!
Read was done 200518_23_N1_00.pnt!
Read was done 210518_00_N1_00.pnt!
Read was done 210518_01_N1_00.pnt!
Read was done 210518_02_N1_00.pnt!
Read was done 210518_03_N1_00.pnt!
Read was done 210518_04_N1_00.pnt!
Read was done 210518_05_N1_00.pnt!
Read was done 210518_06_N1_00.pnt!
Read was done 210518_07_N1_00.pnt!
Read was done 210518_08_N1_00.pnt!
Read was done 210518_09_N1_00.pnt!
Read was done 210518_10_N1_00.pnt!
Read was done 210518_11_N1_00.pnt!
Read was done 210518_12_N1_00.pnt!
Read was done 210518_13_N1_00.pnt!
Read was done 210518_14_N1_00.pnt!
Read was done 210518_15_N1_00.pnt!
Read was done 210518_16_N1_00.pnt!
Read was done 210518

Read was done 300518_13_N1_00.pnt!
Read was done 300518_14_N1_00.pnt!
Read was done 300518_15_N1_00.pnt!
Read was done 300518_16_N1_00.pnt!
Read was done 300518_17_N1_00.pnt!
Read was done 300518_18_N1_00.pnt!
Read was done 300518_19_N1_00.pnt!
Read was done 300518_20_N1_00.pnt!
Read was done 300518_21_N1_00.pnt!
Read was done 300518_22_N1_00.pnt!
Read was done 300518_23_N1_00.pnt!
Read was done 310518_00_N1_00.pnt!
Read was done 310518_01_N1_00.pnt!
Read was done 310518_02_N1_00.pnt!
Read was done 310518_03_N1_00.pnt!
Read was done 310518_04_N1_00.pnt!
Read was done 310518_05_N1_00.pnt!
Read was done 310518_06_N1_00.pnt!
Read was done 310518_07_N1_00.pnt!
Read was done 310518_08_N1_00.pnt!
Read was done 310518_09_N1_00.pnt!
Read was done 310518_10_N1_00.pnt!
Read was done 310518_11_N1_00.pnt!
Read was done 310518_12_N1_00.pnt!
Read was done 310518_13_N1_00.pnt!
Read was done 310518_14_N1_00.pnt!
Read was done 310518_15_N1_00.pnt!
Read was done 310518_16_N1_00.pnt!
Read was done 310518

In [8]:
len(file_with_pulsar)

32

In [9]:
for i in file_with_pulsar[:]:
    print(i)

010518_00_N1_00.pnt
020518_00_N1_00.pnt
030518_00_N1_00.pnt
040518_00_N1_00.pnt
050518_00_N1_00.pnt
060518_00_N1_00.pnt
070518_00_N1_00.pnt
080518_00_N1_00.pnt
080518_23_N1_00.pnt
090518_23_N1_00.pnt
100518_23_N1_00.pnt
110518_23_N1_00.pnt
120518_23_N1_00.pnt
130518_23_N1_00.pnt
140518_23_N1_00.pnt
150518_23_N1_00.pnt
160518_23_N1_00.pnt
170518_23_N1_00.pnt
180518_23_N1_00.pnt
190518_23_N1_00.pnt
200518_23_N1_00.pnt
210518_23_N1_00.pnt
220518_23_N1_00.pnt
230518_23_N1_00.pnt
240518_22_N1_00.pnt
250518_22_N1_00.pnt
260518_22_N1_00.pnt
270518_22_N1_00.pnt
280518_22_N1_00.pnt
290518_22_N1_00.pnt
300518_22_N1_00.pnt
310518_22_N1_00.pnt


In [52]:
 header_pnt[0]

'numpar      16\nsource      source\nalpha       alpha\ndelta       delta\nfcentral    110.25\nwb_total    2.5\ndate_begin  01.05.2018 UTC 01.05.2018\ntime_begin  4:00:00 UTC 0:00:00\ndate_end    01.05.2018\ntime_end    4:59:59\nmodulus      1 2 3 4 5 6\ntresolution 99.9424\nnpoints     36018\nnbands      6\nwbands      0.4150390625 0.4150390625 0.4150390625 0.4150390625 0.4150390625 0.4248046875\nfbands      109.20751953125 109.62255859375 110.03759765625 110.45263671875 110.86767578125 111.28759765625\n'

In [71]:
my_sidereal_time(t_b)

<Angle 5.50369477 hourangle>

In [70]:
may2018

<Time object: scale='utc' format='isot' value=2019-04-21T13:00:00.0000000>

In [57]:
header = {}
for line in header_pnt[0].split('\n'):
    print(line)
    a, *b = line.split()
    header[a] = b

numpar      16
source      source
alpha       alpha
delta       delta
fcentral    110.25
wb_total    2.5
date_begin  01.05.2018 UTC 01.05.2018
time_begin  4:00:00 UTC 0:00:00
date_end    01.05.2018
time_end    4:59:59
modulus      1 2 3 4 5 6
tresolution 99.9424
npoints     36018
nbands      6
wbands      0.4150390625 0.4150390625 0.4150390625 0.4150390625 0.4150390625 0.4248046875
fbands      109.20751953125 109.62255859375 110.03759765625 110.45263671875 110.86767578125 111.28759765625



ValueError: not enough values to unpack (expected at least 1, got 0)

In [58]:
header

{'alpha': ['alpha'],
 'date_begin': ['01.05.2018', 'UTC', '01.05.2018'],
 'date_end': ['01.05.2018'],
 'delta': ['delta'],
 'fbands': ['109.20751953125',
  '109.62255859375',
  '110.03759765625',
  '110.45263671875',
  '110.86767578125',
  '111.28759765625'],
 'fcentral': ['110.25'],
 'modulus': ['1', '2', '3', '4', '5', '6'],
 'nbands': ['6'],
 'npoints': ['36018'],
 'numpar': ['16'],
 'source': ['source'],
 'time_begin': ['4:00:00', 'UTC', '0:00:00'],
 'time_end': ['4:59:59'],
 'tresolution': ['99.9424'],
 'wb_total': ['2.5'],
 'wbands': ['0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4150390625',
  '0.4248046875']}